In [ ]:
import joblib
import pandas as pd
import numpy as np

def predict_conv2d_emission(config: dict, model_path: str, scaler_path: str):
    """
    Dự đoán emissions_rate từ cấu hình Conv2D sử dụng model Lasso đã huấn luyện.

    Args:
        config (dict): {
            "batch_size": int,
            "image_size": int,
            "kernel_size": int,
            "in_channels": int,
            "out_channels": int,
            "stride": int,
            "padding": int
        }
        model_path (str): đường dẫn đến file .joblib chứa mô hình đã lưu
        scaler_path (str): đường dẫn đến file .joblib chứa scaler đã lưu

    Returns:
        float: emissions_rate (kg CO2eq / sec)
    """
    # Tạo DataFrame từ config
    df = pd.DataFrame([config])

    # Thêm các đặc trưng log
    df["log_batch_size"] = np.log1p(df["batch_size"])
    df["log_kernel_size"] = np.log1p(df["kernel_size"])
    df["log_in_channels"] = np.log1p(df["in_channels"])
    df["log_out_channels"] = np.log1p(df["out_channels"])
    df["log_image_size"] = np.log1p(df["image_size"])

    # Load scaler và model
    scaler = joblib.load(scaler_path)
    model = joblib.load(model_path)

    # Lấy đúng các feature đã dùng khi huấn luyện
    feature_names = model.feature_names_in_ if hasattr(model, "feature_names_in_") else df.columns
    X = scaler.transform(df[feature_names])

    # Dự đoán
    emissions_rate = model.predict(X)[0]
    return emissions_rate
